# Importing libraries

In [1]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

# Changing the way pandas displays floats

# Importing dataset

In [2]:
suicides = pd.read_csv('/Users/andressalomferrer/Desktop/ironhack/Projects/Project_3/Project-Week-3-Data-Thieves/your-project/suicides_cleaned.csv')

# QUESTION 4:

# 1. MALES OR FEMALES COMMIT SUICIDE MORE?

In [3]:
suicides.dtypes

Country              object
Year                  int64
Gender               object
Age                  object
Num_Suicides          int64
Population            int64
Suicides/100kPop    float64
GDP/Year              int64
GDP/Capita            int64
Generation           object
dtype: object

In [4]:
suicides = suicides.astype({"Year": 'object'})

In [5]:
# From 1990 till 2015

In [6]:
tot_s_mf = suicides[['Gender','Num_Suicides','Year']].groupby('Gender').sum().reset_index()

In [7]:
tot_s_mf['Percentage'] = tot_s_mf['Num_Suicides'].apply(lambda row: (row/sum(tot_s_mf['Num_Suicides'])*100))

In [8]:
tot_s_mf = tot_s_mf.round({'Percentage': 2})

## 1. Solution: 
## TOTAL SUICIDES MALES AND FEMALES 1990 - 2015:
## males  77.3%, females: 22.7%

# 2. Males or females commit suicide more in 2015?

In [10]:
# To answer this question we are going to create a dataframe for the year 2015:

suicides_2015 = suicides[suicides['Year'] == 2015]

In [11]:
suicides_2015.head(2)

,Country,Year,Gender,Age,Num_Suicides,Population,Suicides/100kPop,GDP/Year,GDP/Capita,Generation
480,Antigua and Barbuda,2015,female,55-74 years,1,6403,15.62,1364863037,14853,Boomers
481,Antigua and Barbuda,2015,female,15-24 years,0,8561,0.00,1364863037,14853,Millenials


In [12]:
# Now we are going to filter by the columns we need:

s_gender2015 = suicides_2015[['Gender','Num_Suicides']].groupby('Gender').sum().reset_index()

## As we can see, the number of suicides in 2015 is higher in males than in females:


In [13]:
s_gender2015

,Gender,Num_Suicides
0,female,47248
1,male,156392


In [14]:
s_gender2015['Percentage'] = s_gender2015['Num_Suicides'].apply(lambda row: (row/sum(s_gender2015['Num_Suicides'])*100))

In [15]:
s_gender2015

,Gender,Num_Suicides,Percentage
0,female,47248,23.201729
1,male,156392,76.798271


In [16]:
s_gender2015 = s_gender2015.round({'Percentage': 2})

# 2. SOLUTION PECENTAGE IN 2015: males 76.8%, females 23.2%

In [133]:
s_gender2015.to_csv('/Users/andressalomferrer/Desktop/ironhack/gender15.csv')

# 3. TOP 10 COUNTRIES BY GENRE:

## BY NUMBER:

### Top 10 countries with more suicides in year 2015 taking into account the number of suicides.

In [18]:
top10countriesbynum = suicides_2015[['Country','Num_Suicides']].groupby('Country').sum().sort_values("Num_Suicides", ascending=False)[0:10]

In [19]:
top10countriesbynum

,Num_Suicides
Country,
United States,44189
Russian Federation,25432
Japan,23092
Republic of Korea,13510
Brazil,11163
Germany,10088
Ukraine,7574
Mexico,6234
Poland,5420


### MALES : TOP 10 COUNTRIES BY NUM

In [20]:
top10malesbynum = suicides_2015[['Country','Gender','Num_Suicides']].loc[suicides_2015['Gender']=='male'].groupby('Country').sum().sort_values("Num_Suicides", ascending=False)[0:10]

In [21]:
top10malesbynum

,Num_Suicides
Country,
United States,33990
Russian Federation,20861
Japan,16146
Republic of Korea,9556
Brazil,8766
Germany,7403
Ukraine,6148
Mexico,4986
Poland,4697


### FEMALES : TOP 10 COUNTRIES BY NUM

In [22]:
top10femalesbynum = suicides_2015[['Country','Gender','Num_Suicides']].loc[suicides_2015['Gender']=='female'].groupby('Country').sum().sort_values("Num_Suicides", ascending=False)[0:10]

In [23]:
top10femalesbynum

,Num_Suicides
Country,
United States,10199
Japan,6946
Russian Federation,4571
Republic of Korea,3954
Germany,2685
Brazil,2397
Ukraine,1426
Mexico,1248
United Kingdom,1178


In [83]:
# Creating an intersection by index to see if the countries with a higher number of suicides
# are the same countries for both genres.

idgnf = pd.Index(top10malesbynum.index)
idgnm = pd.Index(top10femalesbynum.index)

In [84]:
set_intersection = idgnf.intersection(idgnm)

In [85]:
set_intersection

Index(['United States', 'Russian Federation', 'Japan', 'Republic of Korea',
       'Brazil', 'Germany', 'Ukraine', 'Mexico', 'United Kingdom'],
      dtype='object', name='Country')

### As we can see 9 out of 10 countries match

In [27]:
# Spain appears in top10 of countries where females commit suicides.

idgnf.difference(idgnm)

Index(['Spain'], dtype='object', name='Country')

In [28]:
# Poland appears in top10 of countries where males commit suicides.

idgnm.difference(idgnf)

Index(['Poland'], dtype='object', name='Country')

In [29]:
set_intersection = idgnf.intersection(idgnm)

In [30]:
set_intersection

Index(['United States', 'Japan', 'Russian Federation', 'Republic of Korea',
       'Germany', 'Brazil', 'Ukraine', 'Mexico', 'United Kingdom'],
      dtype='object', name='Country')

## Now we realize that we took into account the num of suicides. But if we take into account the size of the population, results can be different. Let's check:


## BY SIZE:

## FROM 1990 - 2015

In [31]:
suicides_rateallyears = suicides[['Country', 'Population', 'Num_Suicides']].groupby('Country').sum()

In [32]:
suicides_rateallyears = suicides_rateallyears.reset_index()

In [33]:
suicides_rateallyears['Rates'] = (suicides_rateallyears['Num_Suicides']/suicides_rateallyears['Population'])*100000


In [34]:
suicides_rateallyears = suicides_rateallyears[['Country','Rates']].groupby('Country').sum().sort_values("Rates", ascending=False)

## SOLUTION TOP10 COUNTRIES WITH THE HIGHEST RATE OF SUICIDE (GENERAL):

In [35]:
suicides_ratesallyears = suicides_rateallyears[0:10]

In [36]:
suicides_ratesallyears

,Rates
Country,
Lithuania,41.464101
Russian Federation,32.959289
Belarus,30.344685
Hungary,30.022407
Sri Lanka,28.872874
Latvia,28.471011
Kazakhstan,26.898614
Slovenia,26.360477
Estonia,25.964525


# Rate of suicide by gender in 2015:

## Male 2015:

In [143]:
males = suicides_2015[suicides_2015['Gender'] == 'male']

In [144]:
males = males[['Country', 'Population', 'Num_Suicides']].groupby('Country').sum()

In [145]:
males['Rates'] = (males['Num_Suicides']/males['Population'])*100000

In [146]:
males = males.reset_index()

In [147]:
males = males[['Country','Rates']].groupby('Country').sum().sort_values("Rates", ascending=False)

In [148]:
malestop10 = males[0:10]

In [151]:
malestop10 = malestop10.round({'Rates': 2})

In [154]:
malestop10

,Rates
Country,
Lithuania,57.67
Republic of Korea,39.34
Latvia,35.79
Slovenia,34.34
Ukraine,33.14
Russian Federation,33.11
Uruguay,32.54
Kazakhstan,31.21
Hungary,31.15


## Female 2015:

In [44]:
females = suicides_2015[suicides_2015['Gender'] == 'female']

In [45]:
females = females[['Country', 'Population', 'Num_Suicides']].groupby('Country').sum()

In [46]:
females['Rates'] = (females['Num_Suicides']/females['Population'])*100000

In [47]:
females = females.reset_index()

In [48]:
females = females[['Country','Rates']].groupby('Country').sum().sort_values("Rates", ascending=False)

In [107]:
femalestop10 = females[0:10]

In [155]:
femalestop10.to_csv('/Users/andressalomferrer/Desktop/ironhack/females15.csv')

In [119]:
# Creating an intersection by index to see which countries appears in both rankings.
set_female = set(femalestop10.index)
set_male = set(malestop10.index)

In [120]:
femalestop10.index

Index(['Republic of Korea', 'Lithuania', 'Japan', 'Belgium', 'Hungary',
       'Croatia', 'Slovenia', 'Latvia', 'Uruguay', 'Norway'],
      dtype='object', name='Country')

In [121]:
countries_intersection = idx1.intersection(idx2)

# Same countries in the top 10 for males and females 2015

In [122]:
countries_intersection

{'Croatia',
 'Hungary',
 'Latvia',
 'Lithuania',
 'Republic of Korea',
 'Slovenia',
 'Uruguay'}

### As we can see 7 out of 10 countries appear in both, by num and by size.

# Different countries for females 2015

In [125]:
females_difference = set_female.difference(set_male)

In [126]:
females_difference

{'Belgium', 'Japan', 'Norway'}

# Different countries for males 2015

In [127]:
males_difference = set_male.difference(set_female)

In [128]:
males_difference

{'Kazakhstan', 'Russian Federation', 'Ukraine'}

# QUESTION 5: 

# 1.  Which generation has committed suicide the most/least?


## First, we will see which generation committed more suicides from 1990 till 2015:

## BY NUMBER:

In [59]:
df_1 = suicides[['Generation','Num_Suicides']].groupby('Generation').sum()

In [156]:
df_1 = df_1.reset_index().sort_values('Num_Suicides', ascending = False)

In [159]:
df_1.sort_values('Num_Suicides', ascending = False)

,index,Generation,Num_Suicides
0,0,Boomers,2164259
1,5,Silent,1573621
2,2,Generation X,1435850
3,4,Millenials,620194
4,1,G.I. Generation,278142
5,3,Generation Z,15906


### We see that in number of suicides, 'Boomers Generation' is the generation than committed the highest number of suicides and Millenials and z generation the lowest . What will happen if we divide the number of suicides per generation by the total population for each generation? Let's check:


## BY SIZE

In [62]:
bysize = suicides[['Generation','Num_Suicides','Population']].groupby('Generation').sum()

In [63]:
bysize['Rates'] = (bysize['Num_Suicides']/bysize['Population'])*100000

In [64]:
bysize = bysize.sort_values('Rates', ascending = False)

In [65]:
bysize = bysize.round({'Rates': 2})

In [66]:
bysize = bysize.drop(['Num_Suicides', 'Population'], axis=1)

In [160]:
bysize.to_csv('/Users/andressalomferrer/Desktop/ironhack/whatever.csv')

## Generation Z is still one of the lowest ones but Silent generation is now one of the biggest one.

# In 2015:

## BY NUMBER:

In [68]:
num_2015 = suicides_2015[['Generation','Num_Suicides']].groupby('Generation').sum()

In [69]:
num_2015 = num_2015.reset_index().sort_values('Num_Suicides', ascending = False)

In [70]:
num_2015

,Generation,Num_Suicides
1,Generation X,71298
3,Millenials,53959
0,Boomers,53588
4,Silent,23114
2,Generation Z,1681


## BY SIZE

In [71]:
size_2015 = suicides_2015[['Generation','Num_Suicides', 'Population']].groupby('Generation').sum()

In [72]:
bysize = suicides_2015[['Generation','Num_Suicides','Population']].groupby('Generation').sum()

In [73]:
bysize['Rates'] = (bysize['Num_Suicides']/bysize['Population'])*100000

In [74]:
bysize = bysize.sort_values('Rates', ascending = False)

In [75]:
bysize = bysize.round({'Rates': 2})

In [76]:
bysize = bysize.drop(['Num_Suicides', 'Population'], axis=1)

In [77]:
bysize

,Rates
Generation,
Silent,19.83
Boomers,15.11
Generation X,13.82
Millenials,9.98
Generation Z,0.68


### In terms of population, we see that Silent Generation committes more suicides followed by the Boomer generation.

# QUESTION 6 Suicide/Population rate per country?

### TOP10 COUNTRIES WITH HIGHER RATE OF SUICIDES FROM 1990-2015:

In [78]:
suicides_hrate = suicides[['Country','Num_Suicides','Population']].groupby('Country').sum()

In [79]:
suicides_hrate['Rates'] = (suicides_hrate['Num_Suicides']/suicides_hrate['Population'])*100000

In [80]:
suicides_hrate = suicides_hrate.sort_values('Rates', ascending = False)[0:10]

In [81]:
suicides_hrate = suicides_hrate[['Rates']]

In [82]:
suicides_hrate

,Rates
Country,
Lithuania,41.464101
Russian Federation,32.959289
Belarus,30.344685
Hungary,30.022407
Sri Lanka,28.872874
Latvia,28.471011
Kazakhstan,26.898614
Slovenia,26.360477
Estonia,25.964525
